In [1]:
import json
import asyncio
from agents import Agent, RunContextWrapper
from agents import set_default_openai_client, set_default_openai_api, set_trace_processors, Runner, trace, gen_trace_id, \
    function_tool
from agents.agent import StopAtTools
from openai.types.responses import EasyInputMessageParam
from phoenix.otel import register
from dotenv import load_dotenv
from agents.models import openai_provider
from openai import AsyncOpenAI
import logging
import os
import ssl
import requests
import numpy as np
import re

from typing_extensions import Any

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv()

ssl._create_default_https_context = ssl._create_unverified_context

PHOENIX_TRACE_URL = os.getenv("PHOENIX_TRACE_URL", 'http://176.99.133.173:6006/v1/traces')
PHOENIX_PROJECT_NAME = os.getenv("PHOENIX_PROJECT_NAME", 'ragas-test')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", '1')
OPENAI_API_URL = os.getenv("OPENAI_API_URL", 'http://176.99.133.173:11434/v1')
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", 'qwen2.5:7b')
RETRIEVER_URL = os.getenv("RETRIEVER_URL", 'http://176.99.133.173:8503/retrieve')

# configure the Phoenix tracer
set_trace_processors([])
tracer_provider = register(
    project_name=PHOENIX_PROJECT_NAME,
    endpoint=PHOENIX_TRACE_URL,
    auto_instrument=True
)

set_default_openai_client(AsyncOpenAI(base_url=OPENAI_API_URL, api_key=OPENAI_API_KEY, timeout=60 * 5))
set_default_openai_api('chat_completions')
openai_provider.DEFAULT_MODEL = DEFAULT_MODEL


def remove_think_tags(text: str) -> str:
    """Удаляет теги <think> и их содержимое из текста.

    Args:
        text: Исходный текст, который может содержать теги <think>.

    Returns:
        Текст без тегов <think> и их содержимого.
    """
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)

class AsyncRunner:
    @staticmethod
    def run_async(func, *args, **kwargs):
        loop = asyncio.get_running_loop()
        try:
            return loop.run_until_complete(func(*args, **kwargs))
        finally:
            loop.close()



@function_tool
def rag_search(wrapper: RunContextWrapper[Any], query: str) -> str:
    """
    Используй для поиска релевантной информации в записях видеоконференций
    :param query: поисковый запрос
    :return: список релевантных документов

    Примеры query:
    1. Как зовут собаку Даши?
    2. К каким выводам пришли в результате обсуждения архитектуры нового автомобиля?
    3. Какие задачи стоят перед командой продаж?
    """
    return rag_search_inner(wrapper, query)


def rag_search_inner(wrapper: RunContextWrapper[Any], query: str) -> str:
    try:
        logging.info(f'rag_search_context: {json.dumps(wrapper.context)}')
        filters = {
            'query': query,
            'classic_search_num': 50,
            'vector_search_num': 50,
            'max_get_num': 5,
            'room_uuid': wrapper.context['room_uuid'] if 'room_uuid' in wrapper.context else ""
        }
        text_objs = requests.post(RETRIEVER_URL, json=filters).json()
        scores = [x['score'] for x in text_objs]
        texts_magic_sort = []
        idx_s = np.argsort(-np.array(scores))

        wrapper.context['ranked_results'] = []

        for idx in idx_s:
            wrapper.context['ranked_results'] = wrapper.context['ranked_results'] + [text_objs[idx]['parent_chunk']]

        splitter = '%$$%#'

        available_room_uuids_strs = set([f"{x['room_uuid']}{splitter}{x['meeting_label']}" for x in text_objs])
        wrapper.context['available_room_uuids'] = []

        for available_room_uuids_str in available_room_uuids_strs:
            room_uuid = available_room_uuids_str.split(sep=splitter)[0]
            meeting_label = available_room_uuids_str.split(sep=splitter)[1]
            wrapper.context['available_room_uuids'].append({
                'room_uuid': room_uuid,
                'meeting_label': meeting_label
            })

        for idx in idx_s:
            doc_string = f"Часть записи видеоконференции c UUID {text_objs[idx]['room_uuid']}:\n{text_objs[idx]['parent_chunk']}"
            if idx % 2 == 1:
                texts_magic_sort = texts_magic_sort + [doc_string]
            else:
                texts_magic_sort = [doc_string] + texts_magic_sort

        context = '\n----\n\n'.join(texts_magic_sort)
        return "Полученные записи видеоконференций:\n\n" + context
    except Exception as e:
        logging.error(e)
        return "Произошла ошибка при запросе к сервису извлечения знаний из видеоконференций"


# Function to run a query with error handling
async def run_agent_query_async(messages, context):
    try:
        agent = Agent(
            name="Videoconference helper",
            instructions="""
Ты помощник по поиску информации и ответам по видеоконференциям.
Используй tool rag_search для нахождения релевантной для ответа информации.
Если релевантной информации не нашлось, то сообщи об этом пользователю.
Отвечай точно на поставленный вопрос и не пиши лишнего что не относится к контексту вопроса.
Перед ответом обязательно вызови rag_search
            """,
            tools=[rag_search]
        )
        trace_id = gen_trace_id()
        with trace(workflow_name="Question Answering", trace_id=trace_id):
            result = await Runner.run(starting_agent=agent, input=messages, context=context)
            return result, trace_id
    except Exception as e:
        logging.error(f"Error processing query: {str(e)}")
        return f"Failed to process query: {str(e)}", None


import nest_asyncio
nest_asyncio.apply()

def run_in_jupyter(messages, context):
    try:
        # Получаем текущий event loop (уже запущенный в Jupyter)
        loop = asyncio.get_event_loop()

        # Запускаем корутину в существующем loop
        if loop.is_running():
            # Если loop уже работает (как в Jupyter), используем create_task
            task = loop.create_task(run_agent_query_async(messages, context))
            # Для простоты ждем завершения (в реальном коде лучше использовать await)
            return loop.run_until_complete(task)
        else:
            # Если loop не работает (как в обычном скрипте)
            return loop.run_until_complete(run_agent_query_async(messages, context))
    except Exception as e:
        logging.error(f"Error in Jupyter runner: {str(e)}")
        return f"Failed to run in Jupyter: {str(e)}", None





🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ragas-test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://176.99.133.173:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [2]:
def get_answer(query):
    context = {'room_uuid': '77779de4-851e-411f-b1e4-acf69268c8b6'}
    result, trace_id = run_in_jupyter(
        [EasyInputMessageParam(role='user', content=query)],
        context=context
    )

    return remove_think_tags(result.final_output), context['ranked_results']

In [3]:
data = [
    {
        "user_input": "Какая книга и какая глава из нее рекомендуются к прочтению для более глубокого понимания английских школ?",
        "answer": "Рекомендуется книга Всеволода Овчинникова «Корни дуба», в частности глава «Старый школьный галстук»."
    },
    {
        "user_input": "Чьи слова о том, что битва при Ватерлоо была выиграна на спортивных площадках Итона, часто повторяют англичане?",
        "answer": "Англичане часто повторяют слова герцога Веллингтонского о том, что битва при Ватерлоо была выиграна на спортивных площадках Итона."
    },
    {
        "user_input": "Какую роль закрытых частных школ подчеркнул герцог Веллингтонский в своей знаменитой фразе?",
        "answer": "Герцог Веллингтонский подчеркнул роль закрытых частных школ в формировании элиты общества."
    },
    {
        "user_input": "Каково основное назначение публичной школы в Англии, согласно тексту?",
        "answer": "Основное назначение такой школы – воспитать джентльмена."
    },
    {
        "user_input": "Каково назначение джентльмена, как его трактует текст?",
        "answer": "Назначение джентльмена — возглавить и повести за собой людей в час трудных испытаний."
    },
    {
        "user_input": "Что парадоксального в названии «публичная школа» (public school) в Англии?",
        "answer": "Парадоксальность названия «публичная школа» в Англии заключается в том, что это не что иное, как частная школа."
    },
    {
        "user_input": "Какое образование давали публичные школы в Англии со средних веков?",
        "answer": "Публичные школы в Англии со средних веков давали классическое образование, необходимое для публичной карьеры, такой как деятельность служителя церкви или государственного чиновника."
    },
    {
        "user_input": "Какой процент от общего числа школьников в настоящее время обучается в публичных школах Англии?",
        "answer": "В настоящее время в подобных учебных заведениях насчитывается менее пяти процентов общего числа школьников."
    },
    {
        "user_input": "Кто и когда ввел новшества, которые сформировали публичные школы в их нынешнем виде?",
        "answer": "Доктор Томас Арнольд, возглавивший публичную школу Регби в 1827 году, ввел новшества, которые сформировали публичные школы в их нынешнем виде."
    },
    {
        "user_input": "Какие новые потребности породили реформы Томаса Арнольда в публичных школах?",
        "answer": "Реформы Томаса Арнольда отражали новые потребности, порожденные ростом империи, которой требовались люди с определенными чертами характера для военной и гражданской службы в заморских владениях, помимо традиционного классического образования."
    },
    {
        "user_input": "Что поставил во главу угла Томас Арнольд при реформировании публичных школ, в отличие от средневековых школ?",
        "answer": "В отличие от средневековых школ, где упор делали на совершенствовании духа через религию, Томас Арнольд поставил во главу угла формирование характера, используя для этого спорт."
    },
    {
        "user_input": "В каком порядке Томас Арнольд перечислил воспитательные цели публичной школы?",
        "answer": "Томас Арнольд перечислил воспитательные цели публичной школы в следующем порядке: во-первых, моральные принципы; во-вторых, джентльменское поведение; и, в-третьих, умственные способности."
    },
    {
        "user_input": "Какие виды спорта доминируют в публичных школах и почему?",
        "answer": "В публичных школах доминируют спортивные игры (командные состязания), так как они считаются эффективным средством воспитания командного духа («team spirit») и умения повиноваться дисциплине."
    },
    {
        "user_input": "Что, по мнению англичан, обретает хороший игрок в составе школьной команды?",
        "answer": "По мнению англичан, хороший игрок в составе школьной команды обретает задатки руководителя и общественного деятеля."
    },
    {
        "user_input": "Как публичные школы видоизменили средневековый рыцарский кодекс чести?",
        "answer": "Публичные школы видоизменили средневековый рыцарский кодекс чести, сделав спортивную этику, понятие «честной игры» («фэйр-плей»), важнейшим нравственным принципом и мерилом порядочности."
    },
    {
        "user_input": "Какие два нововведения Томаса Арнольда, помимо спорта, упоминаются в тексте?",
        "answer": "Томас Арнольд, во-первых, добавил к латыни и розге спорт как рычаг воспитания, а во-вторых, вложил розгу в руки старшеклассников, что по-русски называется «дедовщина». Вторым важным нововведением явилась система старшинства (внутренней субординации)."
    },
    {
        "user_input": "Как организационно устроена публичная школа для углубления субординации?",
        "answer": "Для углубления субординации публичная школа организационно делится не горизонтально (на классы), а вертикально – на «дома», где воспитанники всех классов остаются весь срок обучения."
    },
    {
        "user_input": "Какой самый первый и самый суровый урок преподает публичная школа новичку через старшеклассников?",
        "answer": "Самый первый и самый суровый урок, который публичная школа преподает новичку через старшеклассников, – это необходимость беспрекословно подчиняться всякому, кто по школьной субординации стоит хотя бы на ступеньку выше."
    },
    {
        "user_input": "Какими методами прививают идеи субординации новичкам в английской публичной школе?",
        "answer": "Идеи субординации новичкам прививают не нравоучениями, а унизительными обычаями, наряду с которыми существует вполне официальная система телесных наказаний (розга)."
    },
    {
        "user_input": "Какова роль выпускного (шестого) класса в публичной школе?",
        "answer": "Выпускной (шестой) класс является как бы унтер-офицерским (сержантским) костяком школы; эти подростки отвечают за порядок и вправе применять к младшим дисциплинарные наказания и поощрения."
    },
    {
        "user_input": "Какова основная цель изоляции подростка от семьи и внешнего мира в системе воспитания публичных школ?",
        "answer": "Считается, что лишь совместная жизнь в стенах интерната может привести к тесному и глубокому знанию друг друга, при котором эффективно прививаются и качества подчинённых, и качества руководителей."
    },
    {
        "user_input": "Какие спартанские условия жизни упоминаются как средство закалки в публичных школах?",
        "answer": "В качестве средства закалки упоминаются спартанские условия жизни, в частности, холод и голод, которые воспитывают твёрдость духа, выносливость и самообладание."
    },
    {
        "user_input": "Какой девиз, по мнению автора, мог бы подойти многим публичным школам относительно питания?",
        "answer": "Девизом многих публичных школ поистине могли бы стать слова «Чем хуже питание, тем лучше воспитание»."
    },
    {
        "user_input": "Приведите пример суровых условий в публичной школе Гордонстоун.",
        "answer": "В публичной школе Гордонстоун воспитанники ходят в шортах и принимают холодный душ даже зимой, окна в спальнях держат круглый год открытыми, и не разрешается накрываться больше, чем двумя тонкими одеялами."
    },
    {
        "user_input": "Какую роль выполняют корпоративные галстуки, и особенно галстук публичной школы, в Британии?",
        "answer": "Корпоративные галстуки служат средством социальной классификации. Галстук публичной школы считается наиболее престижным и позволяет судить об образованности, характере, круге знакомств и принадлежности к избранной касте."
    },
    {
        "user_input": "Какая игра слов существует в английском языке, связанная со «школьными галстуками»?",
        "answer": "Выражения «школьные галстуки» и «школьные связи» по-английски звучат одинаково («school tie»), так как слово «tie» означает и «галстук», и «связь» одновременно."
    },
    {
        "user_input": "Какие четыре школы составляют «элиту элит» среди наиболее старых публичных школ?",
        "answer": "Элиту элит среди наиболее старых публичных школ составляют Итон, Винчестер, Рэгби и Харроу."
    },
    {
        "user_input": "Чем Итон отличается от других публичных школ в плане близости к власти и формирования кадров?",
        "answer": "Итон всегда был ближе к Королевскому двору; лично монарх назначает главу Совета попечителей. Из стен Итона вышло восемнадцать премьер-министров. Эта школа больше других напоминает наследственный клуб для политических деятелей."
    },
    {
        "user_input": "Чем Винчестер отличается от Итона в плане отбора учеников и подготовки?",
        "answer": "Винчестеру свойственно уделять больше внимания отбору по способностям, там строже вступительные экзамены. Он дает более основательную подготовку для университета и славится воспитанием «жесткой верхней губы» (самообладания и невозмутимости)."
    },
    {
        "user_input": "О чем в первую очередь думают английские родители, выбирая для сына публичную школу вроде Итона или Винчестера?",
        "answer": "Английские родители думают прежде всего о воздействии, какое окажет публичная школа на характер их сына, о манере поведения, особом выговоре, которые останутся с ним на всю жизнь, а также о друзьях и связях («старый школьный галстук»), которые помогут ему в последующей жизни."
    }
]

In [4]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference, LLMContextPrecisionWithReference

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model=DEFAULT_MODEL, base_url=OPENAI_API_URL, openai_api_key=OPENAI_API_KEY))

context_precision = LLMContextPrecisionWithReference(llm=evaluator_llm)

def run_in_jupyter_LLMContextPrecisionWithReference(sample):
    # Получаем текущий event loop (уже запущенный в Jupyter)
    loop = asyncio.get_event_loop()

    # Запускаем корутину в существующем loop
    if loop.is_running():
        # Если loop уже работает (как в Jupyter), используем create_task
        task = loop.create_task(context_precision.single_turn_ascore(sample))
        # Для простоты ждем завершения (в реальном коде лучше использовать await)
        return loop.run_until_complete(task)
    else:
        # Если loop не работает (как в обычном скрипте)
        return loop.run_until_complete(context_precision.single_turn_ascore(sample))



In [5]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import LLMContextRecall

context_recall = LLMContextRecall(llm=evaluator_llm)


def run_in_jupyter_LLMContextRecall(sample):
    # Получаем текущий event loop (уже запущенный в Jupyter)
    loop = asyncio.get_event_loop()

    # Запускаем корутину в существующем loop
    if loop.is_running():
        # Если loop уже работает (как в Jupyter), используем create_task
        task = loop.create_task(context_recall.single_turn_ascore(sample))
        # Для простоты ждем завершения (в реальном коде лучше использовать await)
        return loop.run_until_complete(task)
    else:
        # Если loop не работает (как в обычном скрипте)
        return loop.run_until_complete(context_recall.single_turn_ascore(sample))

In [6]:
precisions = []
recalls = []
for x in data:
    try:
        print(x['user_input'])
        llm_answer, retrived_chunks = get_answer(x['user_input'])
        sample = SingleTurnSample(
            user_input=x['user_input'],
            response=llm_answer,
            reference=x['answer'],
            retrieved_contexts=retrived_chunks,
        )
        precision = run_in_jupyter_LLMContextPrecisionWithReference(sample)
        recall = run_in_jupyter_LLMContextRecall(sample)
        precisions.append(precision)
        recalls.append(recall)
        print(f'context precision={precision}', f'context recall={recall}')
        print(f'mean context precision={np.mean(precisions)}', f'mean context recall={np.mean(recalls)}')
        
    except Exception as e:
        print(str(e))

Какая книга и какая глава из нее рекомендуются к прочтению для более глубокого понимания английских школ?


2025-05-29 04:37:22,856 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:22,885 - INFO - rag_search_context: {"room_uuid": "77779de4-851e-411f-b1e4-acf69268c8b6"}
2025-05-29 04:37:39,114 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:40,873 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:42,174 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:43,684 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:45,360 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:46,625 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:50,647 - INFO - HTTP Request: POST http://176.99.1

context precision=0.9999999999 context recall=1.0
mean context precision=0.9999999999 mean context recall=1.0
Чьи слова о том, что битва при Ватерлоо была выиграна на спортивных площадках Итона, часто повторяют англичане?


2025-05-29 04:37:51,368 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:37:51,381 - INFO - rag_search_context: {"room_uuid": "77779de4-851e-411f-b1e4-acf69268c8b6"}
2025-05-29 04:38:12,866 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:14,487 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:15,793 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:17,156 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:18,481 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:19,909 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:38:22,860 - INFO - HTTP Request: POST http://176.99.1

The output parser failed to parse the output including retries.
Какую роль закрытых частных школ подчеркнул герцог Веллингтонский в своей знаменитой фразе?


...

context precision=0.99999999995 context recall=1.0
mean context precision=0.7642857142371925 mean context recall=0.8571428571428571
Чем Винчестер отличается от Итона в плане отбора учеников и подготовки?


2025-05-29 04:46:18,149 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"


'ranked_results'
О чем в первую очередь думают английские родители, выбирая для сына публичную школу вроде Итона или Винчестера?


2025-05-29 04:46:18,838 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:18,853 - INFO - rag_search_context: {"room_uuid": "77779de4-851e-411f-b1e4-acf69268c8b6"}
2025-05-29 04:46:44,103 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:45,869 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:47,451 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:48,846 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:50,539 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:51,824 - INFO - HTTP Request: POST http://176.99.133.173:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-29 04:46:57,154 - INFO - HTTP Request: POST http://176.99.1

context precision=0.36666666664833336 context recall=1.0
mean context precision=0.7377777777312686 mean context recall=0.8666666666666667


In [7]:
print("final context precision = ", np.mean(precisions))
print("final context recall = ", np.mean(recalls))

final context precision =  0.7377777777312686
final context recall =  0.8666666666666667
